IMPORTING LIBRARIES

In [1]:
from langchain_groq import ChatGroq

IMPORTING FILE

In [2]:
with open("key.txt", "r") as f:
    api_key=f.readline().strip()

INITIALIZE LLM MODEL

In [3]:
llm = ChatGroq(temperature=0, groq_api_key=api_key, model_name="llama-3.1-70b-versatile")

WEB SCRAPING

In [51]:
from langchain.document_loaders import SeleniumURLLoader

def load_dynamic_page(url):
    loader = SeleniumURLLoader(urls=[url])
    documents = loader.load()
    job_details=documents[0].metadata["description"]
    return job_details

In [52]:
url = "https://walmart.wd5.myworkdayjobs.com/en-US/WalmartExternal/job/IN-KA-BANGALORE-Home-Office-Building-11/SOFTWARE-ENGINEER-III_R-2049883-1"
documents = load_dynamic_page(url)
documents

"Position Summary... Demonstrates up-to-date expertise and applies this to the development, execution, and improvement of action plans by providing expert advice and guidance to others in the application of information and best practices; supporting and aligning efforts to meet customer and business needs; and building commitment for perspectives and rationales. Provides and supports the implementation of business solutions by building relationships and partnerships with key stakeholders; identifying business needs; determining and carrying out necessary processes and practices; monitoring progress and results; recognizing and capitalizing on improvement opportunities; and adapting to competing demands, organizational changes, and new responsibilities. Models compliance with company policies and procedures and supports company mission, values, and standards of ethics and integrity by incorporating these into the development and implementation of business plans; using the Open Door Poli

In [62]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

prompts_extract =PromptTemplate.from_template(
""" 
### SCRAPED TEXT FROM WEBSITE:
{page_data}
### INSTRUCTION:
The scraped text is from the career's page of a wesite.
Your job is to extract the job postings and return them in JSON format containing 
following keys : `role`, `experience`, `skills` and `description`.
Only return the valid json
###(NO PREAMBLE):
"""
)

chain_extract = prompts_extract | llm 

In [63]:
def json_chain_extract(doc):
    res=chain_extract.invoke(input={'page_data':doc})
    json_parser=JsonOutputParser()
    json_res=json_parser.parse(res.content)
    return json_res

In [64]:
json_chain_extract(documents)

{'role': 'Software Engineer',
 'experience': '3 to 7 years',
 'skills': ['Java',
  'Spring Boot',
  'Data Structures',
  'Algorithms',
  'Design Patterns',
  'SQL',
  'Microservices architecture',
  'CI/CD pipeline',
  'Jenkins',
  'Docker',
  'Kubernetes',
  'GCP/Azure/AWS cloud platforms',
  'Kafka',
  'Elastic Search',
  'Kibana',
  'AI/ML',
  'React UI'],
 'description': 'Developing Enterprise scale product and applications, building Distributed Systems, designing and implementing business solutions, collaborating with cross-functional teams, and ensuring business needs are met by building software products using best practices.'}

In [84]:
job=json_chain_extract(documents)

In [85]:
skills=job['skills']

CREATING VECTOR DATABASE FOR PORTFOLIO LINK

In [65]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [67]:
import chromadb
import uuid
client =chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links" : row["Links"]},
                      ids=[str(uuid.uuid4())])

C:\Users\DELL\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████| 79.3M/79.3M [01:13<00:00, 1.12MiB/s]


In [87]:
links =collection.query(query_texts=skills,n_results=2).get('metadatas', [])

In [88]:
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'h

CREATING PROMPT FOR COLD MAIL

In [89]:
email_prompt=PromptTemplate.from_template("""
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """)

In [90]:
chain_generate=email_prompt |llm

In [91]:
email=chain_generate.invoke({"job_description":job, "link_list": lists })

NameError: name 'lists' is not defined